2022-04-15 이주형

# 실시간 감정인식 데모 실행 코드 튜토리얼

라이브러리 import 부분

In [ ]:
import argparse
import torch
import os
import torchvision.models as models
# import torchvision.transforms as transforms
import numpy as np
import model.efficientNetV2 as models
from PIL import Image
import torchvision.datasets as datasets
from utils.transforms import transforms_test as transform
from utils.transforms import NormalizePerImage
import glob
from PIL import Image, ImageFont, ImageDraw
import shutil
from utils.facenet_pytorch import MTCNN
import cv2


## argument 파싱 부분

args.model: 모델웨이트 path 부분\
args.gpu: gpu 번호 명시\
args.resize_h: 0보다 크면 카메라에서 실시간으로 받아진 영상을 args.resize_h로 resize 합니다. default: 0 (resize 안함)\
args.fps: frame per second 입니다.\
***args.entropy_threshold***: 신뢰도와 관련된 숫자이며 ***entropy가 작을수록 신뢰도가 높은 것입니다 (엄격한 기준)***. 디폴트값은 2 이며 상황보며 경험적으로 조절하면 됩니다. 신뢰도가 적으면 추론된 감정을 표기해주지 않습니다.

In [ ]:
parser = argparse.ArgumentParser(description='Emotion inference from cropped face image')
parser.add_argument('--model', type=str, default='checkpoints/efficientNetV2_m.pth.tar')
parser.add_argument('--gpu', type=int, default=0)
parser.add_argument('--resize-h', type=float, default=0)
parser.add_argument('--fps', type=int, default=50)
parser.add_argument('--entropy-threshold', type=float, default=2)

args = parser.parse_args()

EMOTION: 감정을 텍스트로 실시간으로 띄워주는 텍스트

In [ ]:
# EMOTION = ['기쁨', '당황', '분노', '불안', '상처', '슬픔', '중립']
EMOTION =['HAPPY', 'EMBARRASED', 'ANGRY', 'ANXIOUS', 'HURT', 'SAD', 'NEUTRAL']
FONT_SCALE=0.5
FONT_COLOR=(255,255,255)
FONT_THICKNESS=2
normalize=NormalizePerImage()
torch.cuda.set_device(0)

ipcamera 주소입니다.\
ipcamera를 사용하지 않는 경우에 바로 아래의 라인을 사용하시면 됩니다. ```camera = cv2.VideoCapture(0)```

In [ ]:
camera = cv2.VideoCapture('rtsp://192.168.10.101:554/media/1/1')
# camera = cv2.VideoCapture(0)

얼굴인식모델과 감정인식 모델을 가져와서 사용할 디바이스에 카피시키고\
감정인식 모델의 웨이트 파일을 가져와서 감정인식 모델에 로딩시키는 부분입니다.

GPU가 아닌 CPU를 사용할 경우, 주석 처리된 두 라인으로 각라인의 바로 위 라인을 대체시키면 됩니다.

훈련이 아니기 때문에 gradient계산및 필요없는 계산을 하지 않기 위해 eval모드를 켜줍니다.

In [ ]:
mtcnn = MTCNN(image_size=160, margin=8, device=torch.device('cuda:0'))
# mtcnn = MTCNN(image_size=160, margin=8)

efficient_net = models.__dict__['effnetv2_m'](num_classes=7)
efficient_net = efficient_net.cuda(0)
checkpoint = torch.load(args.model, map_location=f'cuda:{args.gpu}')
# checkpoint = torch.load(args.model)
efficient_net.load_state_dict(checkpoint['state_dict'])
efficient_net.eval()

여러 잡다하는 부분들을 해주는 부분입니다.\
카메라의 FPS가 ```args.fps```보다 작을 경우 코드가 돌지 않게 하는 부분은 주석 처리 해놓았습니다. ```# assert(args.fps < FPS)```

In [ ]:
cv2.namedWindow('DEMO')

(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

if (int(major_ver) < 3):
    FPS = camera.get(cv2.CV_CAP_PROP_FPS)
else:
    FPS = camera.get(cv2.CAP_PROP_FPS)

# assert(args.fps < FPS)
# SKIP_FRAME = round(FPS/float(args.fps))

i = 0

실시간 캠을 통한 감정인식이기 때문에 ```while True```를 수행합니다.\
프레임이 제대로 들어왔다면 ```check```가 ```True```값이 됩니다.\
실시간 캠에서 프레임영상을 ```frame```에 저장하고 ```args.resize_h > 0```의 경우, ```frame```을 ```args.resize_h```에 맞게 resize 해줍니다.

In [ ]:
while True:
    i += 1
    # if i == SKIP_FRAME:
    check, frame = camera.read()
    print(f'frame: {frame.shape}')
    if args.resize_h > 0:
        frame = cv2.resize(frame, (int(args.resize_h), int(args.resize_h*frame.shape[0]/float(frame.shape[1]))))

프레임이 제대로 들어온 경우, MTCNN을 통해 얼굴부분을 자른 영상과 (```faces```) bounding box coordinate를 (```boxes```) 가져옵니다.

face가 존재할 경우(```if faces != None```), 그라디언트 계산을 꺼주고 (```with torch.no_grad()```), FP-16 베이스로(```with torch.cuda.amp.autocast()```) face를 normalize한 뒤 gpu로 카피해주고 (```face = normalize(faces).cuda(args.gpu, non_blocking=True)```), 모델에 feed-forward를 수행합니다 (```emotion = efficient_net(torch.unsqueeze(face, 0))```).\
feed-forward에서 나온 logit을 바탕으로 클래스별 확률과(```prob = torch.nn.functional.softmax(emotion)```) 엔트로피(```entropy = -torch.sum(prob * torch.log(prob))```)를 계산합니다. 엔트로피는 신뢰도로 사용할 예정입니다.

opencv를 통해 화면에(```frame```) 박스를 덧입히고 (```cv2.rectangle()```), 인식된 얼굴을 통해 추론된 감정의 신뢰도가 (```entropy```) 기준점보다 높다면 (```< args.entropy_threshold```) 화면에 (```frame```) 높은 신뢰도로 추론된 감정을 텍스트로 덧입힙니다 (```cv2.putText```). 또한 신뢰도가 높을 경우, console창에도 감정을 띄워줍니다 (```print()```지워도 되요).

In [ ]:
    if check:

        # with torch.cuda.amp.autocast():
        faces, boxes = mtcnn(frame)
        if faces != None:
            with torch.no_grad():
                with torch.cuda.amp.autocast():
                    face = normalize(faces).cuda(args.gpu, non_blocking=True)
                    # face = normalize(faces)
                    emotion = efficient_net(torch.unsqueeze(face, 0))
                    prob = torch.nn.functional.softmax(emotion)
                    entropy = -torch.sum(prob * torch.log(prob))
                    # print(f'entropy: {entropy}')
                cv2.rectangle(frame, (int(boxes[0][0]), int(boxes[0][1])), (int(boxes[0][2]), int(boxes[0][3])), (255, 0, 0), 2)
                if entropy < args.entropy_threshold:
                    cv2.putText(frame, EMOTION[torch.argmax(emotion)], (int(boxes[0][0]), int(boxes[0][1])-10), 0, FONT_SCALE, FONT_COLOR, FONT_THICKNESS)

                # draw.rectangle(boxes[0].tolist(), outline=(255, 0, 0), width=6)
                # draw.text((int(boxes[0][0]), int(boxes[0][3])+3), EMOTION[torch.argmax(emotion)], font=ImageFont.truetype("fonts/gulim.ttc", 20), align ="left")
                    print(f'emotion: {EMOTION[torch.argmax(emotion)]}')

얼굴위치박스와 감정 텍스트가 덧입혀진 ```frame```을 화면에 띄워줍니다

In [ ]:
cv2.imshow('DEMO', frame)

args.fps기준으로 기다립니다.

In [ ]:
cv2.waitKey(int(1000/float(args.fps)))

q 를 누르면 꺼집니다. 안꺼지면 ctrl+c 누르면 됩니다.

In [ ]:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

deestructor라고 생각하시면 됩니다.

In [ ]:
camera.release()
cv2.destroyAllWindows()